# Identifying High Risk Areas from NYC Traffic Conditions

<hr/>

*Data Cleaning -*

In [ ]:
# libraries

# pliers
import pandas as pd
import numpy as np
import missingno as msno

# GIS 
import geopandas as gpd
from shapely.geometry import Point, Polygon

# viz
import matplotlib.pyplot as plt

# Data Ingestion

In [4]:
df = pd.read_csv('../data/crash_features_2019.csv')
print(df.shape)
df.head()

(193813, 30)


,DATE,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2019-01-01,01/01/2019,4:10,MANHATTAN,10075.0,40.774967,-73.956800,"(40.774967, -73.9568)",3 AVENUE,EAST 80 STREET,...,Unspecified,NaN,NaN,NaN,4060620,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1,2019-01-01,01/01/2019,14:36,MANHATTAN,10002.0,40.720360,-73.988205,"(40.72036, -73.988205)",NaN,NaN,...,NaN,NaN,NaN,NaN,4060593,Sedan,NaN,NaN,NaN,NaN
2,2019-01-01,01/01/2019,1:41,NaN,NaN,40.661663,-73.923615,"(40.661663, -73.923615)",WINTHROP STREET,NaN,...,Unspecified,NaN,NaN,NaN,4060491,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN
3,2019-01-01,01/01/2019,6:30,BROOKLYN,11219.0,40.635017,-74.002680,"(40.635017, -74.00268)",56 STREET,FORT HAMILTON PARKWAY,...,Unspecified,NaN,NaN,NaN,4061126,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,2019-01-01,01/01/2019,7:03,NaN,NaN,40.868458,-73.821400,"(40.868458, -73.8214)",HUTCHINSON RIVER PARKWAY,NaN,...,Unspecified,NaN,NaN,NaN,4060550,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN


# Create Day of Week Column
<hr/>

In [11]:
# Checking date type

type(df['DATE'][1])

pandas._libs.tslibs.timestamps.Timestamp

In [10]:
# creates datatime for date column

df['DATE'] = pd.to_datetime(df['DATE'])

In [20]:
# creates a columns 

df['dayofweek'] =  df['DATE'].dt.dayofweek

In [26]:
# convert dayofweek to char 

df['dayofweek'] = df['dayofweek'].map ({0:'Mon',1:'Tues',2:'Wed',3:'Thurs',4:'Fri', 5:'Sat',6:'Sun' })

In [32]:
df.iloc[:,-1]

0         Tues
1         Tues
2         Tues
3         Tues
4         Tues
          ... 
193808    Tues
193809    Tues
193810    Tues
193811    Tues
193812    Tues
Name: dayofweek, Length: 193813, dtype: object

# Highways / Expressways / Parkways 
<hr/>

Locating the highways, and creating a boolian column .

In [35]:
# randomly looking through street names to discover highway names

df.iloc[:,8:11].sample(10)

,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME
173532,17 AVENUE,NaN,NaN
65228,36 AVENUE,NaN,NaN
175933,NaN,NaN,146 SOUTH AVENUE
91067,EAST 43 STREET,CLARENDON ROAD,NaN
57489,NaN,NaN,1560 METROPOLITAN AVENUE
54675,21 STREET,40 AVENUE,NaN
72152,NaN,NaN,308 ROGERS AVENUE
88263,FDR DRIVE,NaN,NaN
147129,NaN,NaN,308 EAST 73 STREET
89181,PROSPECT EXPRESSWAY EAST,NaN,NaN


In [36]:
# mask for highways 

highways = "EXPRESSWAY|PKWY|PARKWAY|EXPY|FDR|BQE|EXPWY|HIGHWAY|HENRY HUDSON"

In [37]:
# number of rows where the crash was located on the a 'highway '

df[df['ON STREET NAME'].str.contains(highways, na =False)]['ON STREET NAME'].count()

23869

In [38]:
# a list of the the unique highway names 

np.unique(df[df['ON STREET NAME'].str.contains(highways, na =False)]['ON STREET NAME'])

array(['BAY PARKWAY                     ',
       'BAY RIDGE PARKWAY               ',
       'BELT PARKWAY                    ',
       'BRONX RIVER PARKWAY             ',
       'BROOKLYN QNS EXPRESSWAY         ',
       'BROOKLYN QUEENS EXPRESSWAY      ',
       'BROOKLYN QUEENS EXPRESSWAY E/B  ',
       'BROOKLYN QUEENS EXPRESSWAY RAMP ',
       'BRUCKNER EXPRESSWAY             ',
       'BRUCKNER EXPRESSWAY RAMP        ',
       'CATHEDRAL PARKWAY               ',
       'CHAPIN PARKWAY                  ',
       'CLAREMONT PARKWAY               ',
       'CLEARVIEW EXPRESSWAY            ',
       'CROSS BRONX EXPRESSWAY          ',
       'CROSS BRONX EXPY                ',
       'CROSS BRONX EXPY RAMP           ',
       'CROSS ISLAND PARKWAY            ',
       'CROTONA PARKWAY                 ',
       'DOUGLASTON PARKWAY              ',
       'EAST MOSHOLU PARKWAY NORTH      ',
       'EAST MOSHOLU PARKWAY SOUTH      ',
       'EASTERN PARKWAY                 ',
       'ELL

#### Create a Boolian Column, True crash was on a highway 

In [47]:
#create a Boolian Column, True crash was on a highway 

df['highway'] =  df['ON STREET NAME'].str.contains(highways, na =False)
df['highway'].sample 

<bound method NDFrame.sample of 0         False
1         False
2         False
3         False
4          True
          ...  
193808    False
193809    False
193810     True
193811    False
193812    False
Name: highway, Length: 193813, dtype: bool>

# a look 

In [51]:
df.head()

,DATE,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,dayofweek,highway
0,2019-01-01,01/01/2019,4:10,MANHATTAN,10075.0,40.774967,-73.956800,"(40.774967, -73.9568)",3 AVENUE,EAST 80 STREET,...,NaN,NaN,4060620,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,Tues,False
1,2019-01-01,01/01/2019,14:36,MANHATTAN,10002.0,40.720360,-73.988205,"(40.72036, -73.988205)",NaN,NaN,...,NaN,NaN,4060593,Sedan,NaN,NaN,NaN,NaN,Tues,False
2,2019-01-01,01/01/2019,1:41,NaN,NaN,40.661663,-73.923615,"(40.661663, -73.923615)",WINTHROP STREET,NaN,...,NaN,NaN,4060491,Station Wagon/Sport Utility Vehicle,Sedan,NaN,NaN,NaN,Tues,False
3,2019-01-01,01/01/2019,6:30,BROOKLYN,11219.0,40.635017,-74.002680,"(40.635017, -74.00268)",56 STREET,FORT HAMILTON PARKWAY,...,NaN,NaN,4061126,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,Tues,False
4,2019-01-01,01/01/2019,7:03,NaN,NaN,40.868458,-73.821400,"(40.868458, -73.8214)",HUTCHINSON RIVER PARKWAY,NaN,...,NaN,NaN,4060550,Taxi,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,Tues,True


In [ ]:
# convert to np.... 
    
    # check which rows are missing zip code and borough 
    if (pd.isnull(row['ZIP CODE']) or pd.isnull(row['BOROUGH'])):# and (row['LOCATION']):
        
        # create Point for geo-positioning 
        geo_point = Point(row['LONGITUDE'],row['LATITUDE'])

        # itterates through and pulls zipcode polys to check 
        for ii, zipcode in zipcodes.iterrows():
            
            # checks polys and updates df rows
            if zipcode['geometry'].contains(geo_point):
                row['ZIP CODE'] = zipcode['postalCode']
                row['BOROUGH'] = zipcode['borough']
                df.at[i,'ZIP CODE'] = zipcode['postalCode']
                df.at[i,'BOROUGH'] = zipcode['borough'].upper()
                c += 1
                break

print(f'Number of rows updated {c}')
